# Daum


In [2]:
from bs4 import BeautifulSoup
from selenium import webdriver
import requests
import re
import pickle
import time

In [11]:
#정치(0), 경제(1), 사회(2), 생활/문화(3), 세계(4), 기술/IT(5), 연예(6), 스포츠(7)
class Daum:
    def __init__(self):
        self.options = webdriver.ChromeOptions()
        self.options.add_argument('headless')
        self.driver = ""
        self.url_list =[]
        self.stop_crawling = True
    def Open_Chrome(self, URL) :  
        self.driver = webdriver.Chrome("chromedriver",options = self.options)
        self.driver.get(URL)
        self.driver.implicitly_wait(3)
    def Close_Chrome(self):
        self.driver.close()
        
    def get_url(self,xpath):
        get_xpath= self.driver.find_element_by_xpath(xpath)
        url =  get_xpath.get_attribute('href')
        return url
   
    def loop_get_url(self,limit_num, save_file_name):
        i = 1
        num = len(self.url_list)
        while (i<=15) :
            try :
#                 xpath= '//*[@id="mArticle"]/div[3]/ul/li['  + str(i) + ']/div/strong/a'

#연예, 스포츠 일떄
                xpath= '//*[@id="mArticle"]/div[2]/ul/li['  + str(i) + ']/div/strong/a'
    
                text= self.driver.find_element_by_xpath(xpath).text
                clean = re.sub(r'\s+','',text)
                clean = re.sub('[a-z]|[A-Z]|[0-9]','',clean)
                clean = re.sub('[\{\}\[\]\/?.,;:|\)*~`!^\-_+<>@\#$%&\\\=\(\'\"]','', clean)
                
                if clean != "" :
                    self.url_list.append(self.get_url(xpath))
                    num = num +1
                
                i= i+1
                if(num==limit_num):
                    self.stop_crawling = False
                    break
            except:
                i = i+1
                
        with open(save_file_name, 'wb') as f:
            pickle.dump(self.url_list, f, pickle.HIGHEST_PROTOCOL)
    def pre_day(self) : 
        elem = self.driver.find_element_by_xpath('//*[@id="mArticle"]/div[1]/div/div[1]/a[2]/span')
        elem.click()
        
    def next_page(self,xpath) : 
        elem = self.driver.find_element_by_xpath(xpath)
        elem.click()
 

In [ ]:
#연예
daum = Daum()
url = 'https://media.daum.net/ranking/popular?include=entertain&regDate=20181109'
daum.Open_Chrome(url)
i = 1 
while daum.stop_crawling:
    print("current page : ",i)   
    daum.loop_get_url(1000,'연예.pickle')
    daum.next_page()
    i = i+1
    print("current list length: ",len(daum.url_list))

daum.Close_Chrome()
print(len(daum.url_list))

In [ ]:
#스포츠
daum = Daum()
url = 'https://media.daum.net/ranking/popular?include=sports&regDate=20181109'
daum.Open_Chrome(url)
i = 1 
while daum.stop_crawling:
    print("current page : ",i)   
    daum.loop_get_url(1000,'스포츠.pickle')
    daum.next_page()
    i = i+1
    print("current list length: ",len(daum.url_list))

daum.Close_Chrome()
print(len(daum.url_list))